In [1]:
import dmyplant2
from datetime import datetime, timedelta
import arrow
import pandas as pd
from pprint import pprint as pp
import time
t0 = time.time()
dmyplant2.cred()
mp = dmyplant2.MyPlant(0)
t1 = time.time()
e = dmyplant2.Engine.from_sn(mp, '1486144')
t2 = time.time()
#pp(e.dash)
print(f"Login: {(t1-t0):3.2f}sec\nEngine: {(t2-t1):3.2f}sec")

Login: 0.17sec
Engine: 8.97sec


In [2]:
def get_itemIDs(dat=['Count_OpHour']):
    ret = {}
    for item in dat:
        res = dmyplant2.MyPlant.lookup_dataitems(lookup=item).to_dict('records')[0]
        ret.update({ res.get('id',None) : [res.get('name',None),res.get('unit', '')] })
    return ret

d = ['Various_Values_SpeedAct','Power_PowerAct']
get_itemIDs(d)

{107: ['Various_Values_SpeedAct', 'rpm'], 102: ['Power_PowerAct', 'kW']}

In [3]:
dd = ['Various_Values_SpeedAct','Power_PowerAct']
e.get_dataItems(dd)

{107: ['Various_Values_SpeedAct', 'rpm'], 102: ['Power_PowerAct', 'kW']}

In [4]:
def get_dataItems(e, dat=['Count_OpHour']):
    ret = {}
    for item in dat:
        res = e.get_keyItem(item)
        ret.update({ res.get('id',None) : [res.get('name',None),res.get('unit', '')] })
    return ret
d = ['Various_Values_SpeedAct','Power_PowerAct']
get_dataItems(e, d)

{107: ['Various_Values_SpeedAct', 'rpm'], 102: ['Power_PowerAct', 'kW']}

In [5]:
dmyplant2.Engine._list_cached_validations()

,1486144
Asset ID,159396
Validation Engine,Forsa Hartmoor M01
n,999
oph@start,63.0
serialNumber,1486144
source,from_MyPlant
starts@start,74.0
val start,2022-01-10 00:00:00


In [6]:
e.dash

{'Name': 'Forsa Hartmoor M01',
 'Engine ID': 'M01',
 'Design Number': 'AL87',
 'Engine Type': '624',
 'Engine Version': 'H12',
 'P': 24,
 'P_nom': 4495.0,
 'BMEP': 24.5,
 'serialNumber': '1486144',
 'id': 159396,
 'Count_OpHour': 267.0,
 'val start': Timestamp('2022-01-10 00:00:00'),
 'oph@start': 63,
 'oph parts': 204.0,
 'LOC': 0.14350000000000002}

In [7]:
#r = mp._asset_data_graphQL(assetId=159396)

In [8]:
#res = mp._asset_data(e.serialNumber)

In [9]:
ex = []
ex.append(e['Commissioning Date'])
ex.append('2022-03-28 06:00')
ex.append('2022-03-28 08:14')
ex.append(datetime.now())

ex

['2022-01-10',
 '2022-03-28 06:00',
 '2022-03-28 08:14',
 datetime.datetime(2022, 4, 6, 18, 24, 20, 712975)]

In [10]:
p_from = '2022-03-28 06:00'
p_to = '2022-03-28 08:14'

mes =  e.get_messages(p_from, p_to) #using stored messages.
mes[:3]

,index,assetId,isPrimary,message,name,severity,timestamp,associatedValues
0,818,159396,False,Request module on,1231,600,1648447683434,NaN
1,817,159396,False,Demand oil pump (DC) on,1261,600,1648447683535,NaN
2,816,159396,False,Demand oil pump on,1259,600,1648447683535,NaN


In [11]:
mes[-3:]

,index,assetId,isPrimary,message,name,severity,timestamp,associatedValues
39,779,159396,False,Demand gas leakage check gas train 1,1265,600,1648455196598,NaN
40,778,159396,False,Release from gas leakage check gas train 1,1266,600,1648455206600,NaN
41,0,159396,False,Exhaust gas scavenging flap opened,1275,600,1648455221213,NaN


In [17]:
mes2 =  e.get_messages2(p_from, p_to) #using stored messages.
mes2[:3]

,index,assetId,isPrimary,message,name,severity,timestamp,associatedValues
0,818,159396,False,Request module on,1231,600,1648447683434,NaN
1,817,159396,False,Demand oil pump (DC) on,1261,600,1648447683535,NaN
2,816,159396,False,Demand oil pump on,1259,600,1648447683535,NaN


In [15]:
mes2[-3:]

,index,assetId,isPrimary,message,name,severity,timestamp,associatedValues
39,779,159396,False,Demand gas leakage check gas train 1,1265,600,1648455196598,NaN
40,778,159396,False,Release from gas leakage check gas train 1,1266,600,1648455206600,NaN
41,777,159396,False,Exhaust gas scavenging flap opened,1275,600,1648455221213,NaN


In [21]:
import pandas as pd
class Start_Data_Collector:
    def __init__(self):
        self._vset = []
        self.start = None
        self.end = None
        self._data = pd.DataFrame([])

    @property
    def data(self):
        return self._data

    def left_upper_edge(self, v, data, factor, xmax, ymax):
        self._data = data[((data.datetime >= self.start) & (self.datetime <= self.end))]
        x0 = self._data.iloc[0]['datetime']
        y0 = 0
        x1 = self._data.iloc[-1]['datetime']
        y1 = max(self._data[v]) * factor
        self._data['helpline'] = self._data[v]  + (x0 - self._data['datetime'])* (y1-y0)/(x1-x0) + y0
        point = self._data['helpline'].idxmax()
        if point == point: # test for not NaN
            edge = self._data.loc[point]
            xmax = edge['datetime']
            ymax = self._data.at[edge.name,v]
        return xmax, ymax

    def collect(self, startversuch, result, data):
        pass

    def register(self, startversuch, vset, tfrom, tto):
        return vset, tfrom, tto

class Target_load_Collector(Start_Data_Collector):
    def __init__(self, ratedload, period_factor=3, helplinefactor=0.8):
        super().__init__()
        self._vset += ['Power_PowerAct']
        self.ratedload = ratedload
        self.period_factor=period_factor
        self.helplinefactor=helplinefactor

    def collect(self,startversuch ,results, data):
        xmax = startversuch['endtime']
        ymax = 0.0
        if 'loadramp' in startversuch['startstoptiming']:
            if not data.empty:
                xmax, ymax = self.left_upper_edge('Power_PowerAct', data, self.helplinefactor, xmax, ymax)
        duration = (xmax - self.start).seconds
        ramprate = ymax / duration
        if  duration < 5: # constant load ?
            xmax = startversuch['endtime']
            ymax = 0.0

        sno = startversuch['no']
        results['starts'][sno]['startstoptiming']['loadramp'][0]['end'] = xmax
        if 'targetoperation' in self.results['starts'][sno]['startstoptiming']:
            results['starts'][sno]['startstoptiming']['targetoperation'][0]['start'] = xmax
        results['starts'][sno]['targetload'] = ymax
        results['starts'][sno]['ramprate'] = ramprate / self.ratedload * 100.0
        return results

    def register(self,startversuch,vset,tfrom,tto):
        vset += self._vset
        if 'loadramp' in startversuch['startstoptiming']:
            self.start = startversuch['startstoptiming']['loadramp'][-1]['start']
            tfrom = self.start if self.start < tfrom else tfrom
            self.end = startversuch['startstoptiming']['loadramp'][-1]['end']
            self.end = self.start + self.period_factor * (self.end-self.start)
            tto = self.end if self.end > tto else tto
        return vset, tfrom, tto


result = {}
vset = []
tfrom = pd.to_datetime('2021-12-01')
tto = pd.to_datetime('2022-06-01')
startversuch = {
    'startstoptiming': {
        'loadramp' : [
            {'start': pd.to_datetime('2022-01-01'), 
            'end': pd.to_datetime('2022-02-01') }
        ] 
    }
}

t_reach = Target_load_Collector(3, 0.8)
vset, tfrom, tto = t_reach.register(startversuch, vset, tfrom=tfrom, tto=tto)
vset,tfrom,tto

(['Power_PowerAct'],
 Timestamp('2021-12-01 00:00:00'),
 Timestamp('2022-06-01 00:00:00'))